## ⚡ Упр. - Установка и настройка SQLAlchemy

In [1]:
# !pip install sqlalchemy

In [2]:
# Драйвер для PostgreSQL
# !pip install psycopg2-binary 

## ⚡ Упражнения - SQLAlchemy
 - Создайте модель данных для таблицы `products` с полями: `id`, `name`, `price` и `category_id`.
 - Напишите функцию, которая добавляет новый продукт в базу данных.
 - Реализуйте функцию, которая получает все продукты определенной категории.
 - Добавьте в модель products отношение с моделью categories (один-ко-многим).


In [3]:
# подключаем модули
from sqlalchemy import (create_engine, Column, Integer, Float,
                        String, DateTime, DECIMAL, ForeignKey)
from sqlalchemy.sql import func
from sqlalchemy.orm import declarative_base, relationship, sessionmaker
from sqlalchemy import inspect

#### Подключение к базе данных.
Чтобы не светить пароль - он записан в файле .env  тамже записаны данные о пользователе и прочее, что хотели бы скрыть.
такой файл добавлен в гитигноре.   
пароль хранится локально при этом остается неизвестен широкой публике.

![](../images/11_01.png)<br>


In [4]:
# загружаем модуль
from dotenv import load_dotenv
#грузим пароли запуская поиск файла .env локально
#поиск файла .env и загрузка из него переменныx среды - файл в папке с тетрадкой д.лежать +в гитигноре
load_dotenv()

True

In [5]:
# импортируем модуль для взаимодействия с файлами и папками на жестком диске.
import os
# получаем параметры из хранимого в специальном скрытом файле .env - не храним пароли в явном виде
db_config = {
    'user': os.getenv('USERNAME'),#имя пользователя
    'pwd': os.getenv('PASSWORD'), #пароль  подгружается из локального файла
    'host': os.getenv('HOST'),  
    'port': 5432, #порт подключения
    # 'db': os.getenv('DATABASE') #название базы данных
    'db': 'report' #название базы данных
}
# собираем коннектор
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

#  Создание движка SQLAlchemy
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

#### Создание моделей данных

In [6]:
Base = declarative_base()

class Category(Base):
    __tablename__ = 'categories'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    products = relationship('Product', back_populates='category') # здесь указываем отношение ко многим


class Product(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    price = Column(DECIMAL(10, 2), nullable=False)
    cost = Column(DECIMAL(10, 2), nullable=False)
    category_id = Column(Integer, ForeignKey('categories.id'), nullable=False)
   
    category = relationship('Category', back_populates='products') # здесь указываем отношение ко многим

    def __repr__(self):
        return (
            f'<Product(id={self.id}, name="{self.name}", '
            f'price={self.price}, cost={self.cost})>'
        )

Session = sessionmaker(bind=engine)
session = Session()

#### Функция для добавления нового продукта. 
Получает данные по продукту и объект сессии.

In [7]:
def add_product(session, name, price, cost, category_id):
    new_product = Product(
        name=name,
        price=price,
        cost=cost,
        category_id=category_id,
         )
    session.add(new_product)
    session.commit()
    print(f'Продукт "{name}" успешно добавлен!.')

#### Добавляем продукт

In [ ]:
add_product(
    session,
    name='Новый продукт',
    price=1000,
    cost=500,
    category_id=1
)

#### Функция для получения всех продуктов заданной категории

In [ ]:
def get_products_by_category(session, category_name):
    try:
        category = session.query(Category).filter(
            Category.name == category_name).one()
        return category.products
    except Exception as e:
        print(e)
        return None

products = get_products_by_category(session, 'Одежда')
print(*products, sep='\n')
